In [ ]:
import cv2
import opencv_jupyter_ui as jcv2
from feat import Detector
from IPython.display import Image


from feat.utils import FEAT_EMOTION_COLUMNS

In [ ]:
detector = Detector(device="cuda")

In [ ]:
import os

# Construct the path to the image folder
ori_folder_path = os.path.join( '..', 'dataset', 'images')


# load all the images in the folder
images_filename = os.listdir(ori_folder_path)

# AU codes
predicted_aus = ["AU1","AU2","AU4","AU5","AU6","AU7","AU9","AU10","AU11","AU12","AU14","AU15",
                    "AU17","AU20","AU23","AU24","AU25","AU26","AU28", "AU43"]

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['file', 'face']+predicted_aus)

In [ ]:
# loop through all the images
for image_filename in images_filename:
    frame = cv2.imread(ori_folder_path + "/" + image_filename)
    faces = detector.detect_faces(frame)
    landmarks = detector.detect_landmarks(frame, faces)
    emotions = detector.detect_emotions(frame, faces, landmarks)
    AUs = detector.detect_aus(frame, landmarks)
    faces = faces[0]
    landmarks = landmarks[0]
    emotions = emotions[0]
    AUs = AUs[0]

    strongest_emotion = emotions.argmax(axis=1)

    # draw rectangles around the faces
    for (face, top_emo) in zip(faces, strongest_emotion):
        (x0, y0, x1, y1, p) = face
        cv2.rectangle(frame, (int(x0), int(y0)), (int(x1), int(y1)), (255, 0, 0), 3)
        cv2.putText(frame, FEAT_EMOTION_COLUMNS[top_emo], (int(x0), int(y0 - 10)), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 2)
    
    # save each AUs to the dataframe
    for i in range(len(faces)):
        # remove .jpg from the image name
        filename = image_filename[:-4]
        row = [filename, i]
        row.extend(AUs[i])
        print(row)
        df.loc[len(df)] = row

    
    
    results_file_path = os.path.join('..', 'processed', 'images', image_filename)
    cv2.imwrite(results_file_path, frame)
# save the results to a csv file
results_file_path = os.path.join('..', 'processed', 'aus.csv')
df.to_csv(results_file_path, index=False)

In [ ]:
# accoring to dataset/annotations.csv, separate the images into positive and negative

# load the annotations
annotations = pd.read_csv(os.path.join('..', 'dataset', 'annotations.csv'))
positives = df[df['file'].isin(annotations[annotations['valence'] == 'positive']['file'])]
negatives = df[df['file'].isin(annotations[annotations['valence'] == 'negative']['file'])]

# For each AU, calculate the absolute difference between the positive mean and the negative mean
diffs = {}
for au in predicted_aus:
    pos_mean = positives[au].mean()
    neg_mean = negatives[au].mean()
    diff = abs(pos_mean - neg_mean)
    diffs[au] = diff

# sort the AUs by the difference
sorted_diffs = sorted(diffs.items(), key=lambda x: x[1], reverse=True)
print(sorted_diffs)


In [ ]:
# au_visualization.png
# The x axis must correspond to which AU is being plotted, in the sorted order. Each data point must be correctly labeled with the name of the AU.
# The y axis must correspond to the absolute difference of means for that AU.
# By default, plt.plot() connects all the points as if they belonged to a continuous function. 
# Given the discrete nature of the x axis, we recommend using individual markers to display each data point
import matplotlib.pyplot as plt
import numpy as np

# get the AUs and the differences
aus, diffs = zip(*sorted_diffs)

# plot the AUs and the differences
plt.figure(figsize=(10, 5))
plt.plot(aus, diffs, marker='o', linestyle='None')
plt.xlabel('Action Units')
plt.ylabel('Absolute Difference of Means')
plt.title('Absolute Difference in AU Means (Pos. vs Neg.)')
au_visualization_path = os.path.join('..', 'processed', 'au_visualization.png') 
plt.savefig(au_visualization_path)


In [ ]:
print(FEAT_EMOTION_COLUMNS)